In [2]:
from microfaune.detection import RNNDetector
import csv
import os
import glob
import pandas as pd
import numpy as np
from microfaune import audio
import scipy.signal as scipy_signal
from IPython.display import clear_output

In [3]:
labelsFilePath = "" #Fill this in
dataset = pd.read_csv(labelsFilePath)
dataset["Microfaune_Global_Score_Retrained"] = np.nan
dataset

,Folder,Clip,Bird(s)?,Microfaune_Global_Score_Retrained
0,WWF5_samples_split,5D2C0D00_5.WAV,y,NaN
1,WWF5_samples_split,5D208AE8_13.WAV,n,NaN
2,WWF5_samples_split,5D28A2C8_0.WAV,n,NaN
3,WWF5_samples_split,5D208AE8_10.WAV,y,NaN
4,WWF5_samples_split,5D2C0D00_8.WAV,y,NaN
...,...,...,...,...
7115,AM1_samples_split,20190627_151000_12.WAV,y,NaN
7116,AM1_samples_split,20190701_084000_1.WAV,n,NaN
7117,AM1_samples_split,20190624_074000_14.WAV,n,NaN
7118,AM1_samples_split,20190621_085000_19.WAV,n,NaN


In [4]:
weightsFilePath = "" #Fill this in
dataPath = "" #Fill this in (Audiomoth data path)
Normalized_Sample_Rate = 44100
bird_detector = RNNDetector(weightsFilePath)
errCount = 0
counter = 0
total = str(len(dataset))
for ndx in dataset.index:
    folder_name = dataset["Folder"][ndx]
    clip_name = dataset["Clip"][ndx]
    clip_path = dataPath + folder_name + '/' + clip_name
    
    SAMPLE_RATE, SIGNAL = audio.load_wav(clip_path)

    # downsample the audio if the sample rate > 44.1 kHz
    # Force everything into the human hearing range.
    # May consider reworking this function so that it upsamples as well
    if SAMPLE_RATE > Normalized_Sample_Rate:
        rate_ratio = Normalized_Sample_Rate / SAMPLE_RATE
        SIGNAL = scipy_signal.resample(
                SIGNAL, int(len(SIGNAL)*rate_ratio))
        SAMPLE_RATE = Normalized_Sample_Rate

    # convert stereo to mono if needed
    if len(SIGNAL.shape) == 2:
        SIGNAL = SIGNAL.sum(axis=1) / 2

    # detection
    try:
        microfaune_features = bird_detector.compute_features([SIGNAL])
        global_score, local_score = bird_detector.predict(microfaune_features)
    except:
        print("Error in detection, skipping", dataset["Clip"][ndx])
        errCount += 1
        continue
    counter += 1
    clear_output(wait=True)
    print(str(counter) + "/" + total)
    print("Errors: " + str(errCount))
    
    dataset["Microfaune_Global_Score_Retrained"][ndx] = global_score[0][0]
print("Number of errors: " + str(errCount))

7120/7120
Errors: 0
Number of errors: 0


In [5]:
dataset.columns = ["Folder", "Clip", "Human_Bird_Label", "Microfaune_Global_Score"]
dataset

,Folder,Clip,Human_Bird_Label,Microfaune_Global_Score
0,WWF5_samples_split,5D2C0D00_5.WAV,y,0.596345
1,WWF5_samples_split,5D208AE8_13.WAV,n,0.181447
2,WWF5_samples_split,5D28A2C8_0.WAV,n,0.066197
3,WWF5_samples_split,5D208AE8_10.WAV,y,0.111238
4,WWF5_samples_split,5D2C0D00_8.WAV,y,0.293925
...,...,...,...,...
7115,AM1_samples_split,20190627_151000_12.WAV,y,0.456806
7116,AM1_samples_split,20190701_084000_1.WAV,n,0.140950
7117,AM1_samples_split,20190624_074000_14.WAV,n,0.078280
7118,AM1_samples_split,20190621_085000_19.WAV,n,0.302579


In [6]:
dataset.to_csv("StratifiedRT2.csv")